# 💻 Geopandas: en introduksjon

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GMGI221-2024/forelesninger/blob/main/04_geopandas.ipynb)

I denne seksjonen vil vi dekke det grunnleggende med *geopandas*, et Python-bibliotek for
å samhandle med romlig vektordata.

[Geopandas](https://geopandas.org/) gir et brukervennlig grensesnitt til vektordatasett. Det kombinerer mulighetene til *pandas*
med geometrikapabilitetene til
[shapely](#02_geometriske_objekter), [romlig-filformateringstøtte
fra fiona](#03_vektor) og kartprojeksjonsbibliotekene til
pyproj(som vi ser på neste uke).

Hoveddatastrukturene i geopandas er `GeoDataFrame`s og `GeoSeries`. De
utvider funksjonaliteten til `pandas.DataFrame`s og `pandas.Series`.

Det er en nøkkelforskjell mellom pandas dataframes og geopandas
[`GeoDataFrame`s](https://geopandas.org/en/stable/docs/user_guide/data_structures.html#geodataframe):
en `GeoDataFrame` inneholder en ekstra kolonne for geometrier. Som standard er
navnet på denne kolonnen `geometry`, og det er en
[`GeoSeries`](https://geopandas.org/en/stable/docs/user_guide/data_structures.html#geoseries)
som inneholder geometrier (punkter, linjer, polygoner, ...) som
`shapely.geometry` objekter.

In [ ]:
import pathlib
import geopandas
import numpy

DATA_MAPPE = pathlib.Path().resolve() / "data"

HIGHLIGHT_STYLE = "background: #f66161;"

# så følgende blokk er litt dårlig magi for å få tabellutdataen til å se
# fin ut (denne cellen er skjult, vi er bare interessert i en kort tabellisting
# der geometrikolonnen er fremhevet).
#
# For dette, vi
#    1. konverterer geopandas tilbake til en ‘normal’ pandas.DataFrame med en forkortet
#       WKT-streng i geometrikolonnen
#    1b. samtidig som vi gjør det, blir vi kvitt de fleste av kolonnene (gir de gjenværende kolonner nye navn)
#    2. bruker stilen på alle celler i kolonnen "geometry", og til aksen-1-indeksen "geometry"

# Hvorfor gikk jeg via en ‘plain’ `pandas.DataFrame`?
# `pandas.set_option("display.max_colwidth", 40)` ble ignorert, så dette så ut som den reneste måten

df = geopandas.read_file(DATA_MAPPE / "arealdekke" / "ArealdekkeN50.gpkg")

df["geom"] = df.geometry.to_wkt().apply(lambda wkt: wkt[:40] + " ...")

df = df[["klasse", "klasse_navn", "geometry"]]

(
    df.head().style
        .map(lambda x: HIGHLIGHT_STYLE, subset=["geometry"])
        .apply_index(lambda x: numpy.where(x.isin(["geometry"]), HIGHLIGHT_STYLE, ""), axis=1)
)

## Inputdata: Arealdekke over Ås kommune

I denne notebooken skal vi jobbe med et modifisert datasett fra [Kartverkets N50-serie](https://www.kartverket.no/api-og-data/kartgrunnlag-fastlands-norge).

Til denne notebooken har vi lastet ned Arealdekke-data og endret det noe for å passe til formålet for denne timen. Dataene er lastet ned fra [Geonorge](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac). Filen vi skal jobbe med ligger i `data/ArealdekkeN50.gpkg`.

---

## Les og utforsk romlig datasett

Før vi prøver å laste inn noen filer, la oss ikke glemme å definere en konstant
som peker til vår datamappe:

In [ ]:
import pathlib 


I denne notebooken skal vi fokusere på arealdekke-klasser

**Målet vårt i denne notebooken er å lagre alle arealdekke-klassene i separate filer**.

*Arealdekke-klasser i den datasettet:*

|  Klasse | Navn på klasse   
|----------------|-----------
| 100          | myr
| 110          | steinbrudd
| 120          | tettbebyggelse
| 200          | innsjø
| 300          | indstriområde
| 400          | havflate
| 500          | gravplass
| 600          | dyrket mark
| 700          | åpent område
| 800          | skog
| 900          | sportidrettplass

:::{admonition} Søk etter filer ved hjelp av et mønster
:class: hint


Først, sjekk datatype av det leste datasettet:

Alt gikk bra, og vi har en `geopandas.GeoDataFrame`. 
La oss også utforske dataene: (1) skriv ut de første få radene, og 
(2) list opp kolonnene.

Dette datasettet har flere kolonner enn vi trenger, la oss gjøre et utvalg av de vi trenger. Vi beholder 'klasse’ og ’klassenavn’, og selvfølgelig kolonnen `geometry`.

Hvordan ser datasettet ut nå?

:::{admonition} Sjekk din forståelse:
:class: hint

Bruk dine pandas ferdigheter på dette geopandas datasettet for å finne ut følgende
informasjon:

- Hvor mange rader har datasettet?
- Hvor mange unike klasser?
:::




---

### Utforsk datasettet på et kart:

Som geografer, elsker vi kart. Men utover det, er det alltid en god idé å
utforske et nytt datasett også på et kart. For å lage et enkelt kart av en
`geopandas.GeoDataFrame`, bruk ganske enkelt dens `plot()` metode. Den fungerer likt som
i pandas, men **tegner et kart basert på geometriene i datasettet** i stedet for et diagram.

Voilá! Det er faktisk så enkelt å lage et kart ut av et geografisk datasett.
Geopandas posisjonerer automatisk kartet ditt på en måte som dekker hele
utstrekningen av dataene dine.

### Geometrier i geopandas

Geopandas drar nytte av shapelys geometriobjekter. Geometrier lagres
i en kolonne kalt *geometry*.

La oss skrive ut de første 5 radene av kolonnen `geometry`:

Kolonnen `geometry` inneholder kjente verdier:
*Well-Known Text* (WKT) strenger. La deg ikke lure, de er faktisk,
`shapely.geometry` objekter (du husker de kanskje fra [forrige uke](#02_geometriske_objekter)) som når man bruke `print()` eller konverterer til
en `str`, blir representert som en WKT-streng).

Siden geometriene i en `GeoDataFrame` er lagret som shapely-objekter, kan vi
bruke **shapely metoder** for å håndtere geometrier i geopandas.

La oss ta en nærmere titt på (en av) polygon-geometriene i datasettet,
og prøve å bruke noe av shapely-funksjonaliteten vi allerede er kjent med. For enkelhetens skyld, jobber vi først med geometrien til bare den aller første linjen:

In [ ]:
# Verdien av kolonnen `geometry` i rad 0:


In [ ]:
# Skriv ut informasjon om arealet 
print(f"Område: {} m².")

:::{admonition} Områdemålenhet
:class: note

Her kjenner vi koordinatsystemet (CRS) til inputdatasettet. CRS definerer også måleenheten (i vårt tilfelle, meter). Derfor kan vi skrive ut det beregnede området, inkludert en områdemåleenhet (kvadratmeter).
:::


La oss gjøre det samme for flere rader, og utforske ulike måter å gjøre det på.
Først bruker vi  `iterrows()`-metoden:

In [ ]:
# Iterer over de første 5 radene i datasettet


Som du ser er alle **pandas** funksjoner, som `iterrows()`-metoden, tilgjengelige i geopandas uten behov for å kalle pandas separat. Geopandas bygger på toppen av pandas, og arver mesteparten av funksjonaliteten.

Selvfølgelig er ikke `iterrows()`-metoden den mest praktiske og effektive måten
å beregne arealet til mange rader. Både `GeoSeries` (geometri-kolonner) og
`GeoDataFrame`s har en `area`-egenskap:

In [ ]:
# `area`-egenskapen til en `GeoDataFrame`


In [ ]:
# `area`-egenskapen til en `GeoSeries`


Det er enkelt å lage en ny kolonne som holder arealet:

<div style="border: 1px solid #ccc; padding: 10px; border-radius: 5px; background-color:rgb(177, 226, 250); color:#000;">
<strong>Spørsmål:</strong><br>
Kjør cellen under for å svare på spørsmålet:
</div>

In [1]:
from IPython.display import IFrame

IFrame("https://haavard-polling.vercel.app/answer/2e4018d5-d2cf-4c20-b79f-6c1f9c996966", width=800, height=600)

:::{admonition} Beskrivende statistikk
:class: hint

Vet du hvordan du beregner *minimum*, *maksimum*, *sum*, *gjennomsnitt*, og
*standardavvik* av en pandas-kolonne? ([Les mer her, hvis du trenger å friske opp Pandas-kunnskapene dine](https://pythongis.org/part1/chapter-03/nb/00-pandas-basics.html#descriptive-statistics))
Hva er disse verdiene for arealkolonnen i datasettet?
:::



## Lagre en delmengde av data til en fil

[Tidligere](#03_vektor), har vi lært
hvordan vi skriver en hel `GeoDataFrame` til en fil. Vi kan også skrive en
filtrert delmengde av et datasett til en ny fil, f.eks. for å hjelpe med behandlingen av komplekse datasett.

Først, isoler innsjøene i inngangsdatasettet (klassenummer `200`, se tabell
over):

Deretter, tegn datadelmengden for å visuelt sjekke om den ser riktig ut:

Og til slutt, skriv de filtrerte dataene til en Shapefile:

Sjekk [Vector Data I/O](#03_vektor) avsnittet for å se hvilke dataformater
geopandas kan skrive til.

## Gruppering av data

En spesielt nyttig metode i (geo)pandas' dataframes er deres grupperingsfunksjon: [`groupby()`](https://pandas.pydata.org/docs/user_guide/groupby.html)
kan **dele data inn i grupper** basert på noen kriterier, **bruke** en funksjon
individuelt til hver av gruppene, og **kombinere** resultater av en slik
operasjon i en felles datastruktur.

Vi kan bruke *gruppering* her for å dele inputdatasettet vårt i delmengder som relatere
til hver av `klasse`ne i arealdekke, deretter lagre en separat fil for hver
klasse.

La oss starte dette ved igjen å ta en titt på hvordan datasettet faktisk ser ut:

Husk: kolonnen `klasse` inneholder informasjon om en polygons arealbrukstype. Bruk metoden [`pandas.Series.unique()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html) for å liste alle verdier som forekommer:

For å gruppere data, bruk data-rammens `groupby()` metode, oppgi et kolonnenavn som parameter:

Så, `gruppert_data` er et `DataFrameGroupBy` objekt. Inne i et `GroupBy` objekt,
er egenskapen `groups` en ordbok som fungerer som en oppslagstabell: den registrerer
hvilke rader som hører til hvilken gruppe. Nøklene i ordboken er de unike
verdiene av gruppekolonnen:

Imidlertid kan man også ganske enkelt iterere over hele `GroupBy` objektet. La oss
telle hvor mange rader med data hver gruppe har:

Det er for eksempel 80 innsjøpolygoner (klasse `200`) i inngangsdatasettet.

For å få alle rader som tilhører en bestemt gruppe, bruk `get_group()`
metoden, som returnerer en helt ny `GeoDataFrame`:

:::{caution}OBS
Indeksen i den nye data-frammen forblir den samme som i det ugrupperte inputdatasettet.
Dette kan være nyttig, for eksempel når du vil slå sammen de grupperte dataene
tilbake til de originale inputdataene.
:::


## Skriv grupperte data til separate filer

Nå har vi alle nødvendige verktøy for hånden for å dele inputdataene i
separate datasett for hver arealdekkeklasse, og skrive de individuelle delmengdene til
nye, separate, filer. Faktisk ser koden nesten for enkel ut, gjør den ikke?

In [ ]:
# Iterer over inngangsdataene, gruppert etter "klasse"
for key, group in data.groupby("klasse"):
    # lagre gruppen til en ny shapefile
    

:::{admonition} Filnavn
:class: attention

Vi brukte en `pathlib.Path` kombinert med en f-streng for å generere den nye output-filens sti og navn. Sjekk [Håndtering av filstier-notebooken](#03_filstier)
for å gå gjennom hvordan de fungerer.
:::


## Ekstra: lagre sammendragsstatistikk til CSV regneark

Når resultatene av en operasjon på en `GeoDataFrame` ikke inkluderer en
geometri, vil data-rammen som kommer ut automatisk bli en 'vanlig'
`pandas.DataFrame`, og kan lagres til standard tabellformater.

En interessant anvendelse av dette er å lagre grunnleggende beskrivende statistikk av et geografisk datasett til en CSV-tabell. For eksempel ønsker vi kanskje å vite arealet hver arealdekkeklasse dekker.

Igjen starter vi med å gruppere inputdataene etter arealdekke, og deretter beregne summen av hver klasses areal. Dette kan kondenseres til en linje med kode:

Vi kan deretter lagre den resulterende tabellen til en CSV-fil ved å bruke  standard pandas tilnærming.